# Experiment with the `Inversion` class

In [1]:
import numpy as np

from inversion_ideas import (
    DataMisfit,
    TikhonovZero,
    ConjugateGradient,
    Inversion,
    MultiplierCooler,
    InversionLog,
    ChiTarget,
    create_inversion,
)

from regressor import LinearRegressor

## Create a true model and synthetic data for a linear regressor

In [2]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

array([0.78225148, 0.67148671, 0.2373809 , 0.17946133, 0.34662367,
       0.15210999, 0.31142952, 0.23900652, 0.54355731, 0.91770851])

In [3]:
# Build the X array
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [4]:
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
noise = rng.normal(scale=1e-2 * maxabs, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

array([2.83840696, 2.18091081, 2.00623242, 2.08333039, 2.01694883,
       2.7826232 , 2.10564027, 1.27333506, 2.08859855, 1.94177648,
       1.88492037, 2.92394733, 2.17231952, 3.08009275, 1.61670886,
       1.77403753, 2.67305005, 1.91413882, 2.42117827, 2.13991628,
       2.0153805 , 2.71388471, 2.65944255, 2.44416121, 3.14217523])

## Inversion

In [5]:
uncertainty = 1e-2 * maxabs * np.ones_like(synthetic_data)
simulation = LinearRegressor(X)
data_misfit = DataMisfit(synthetic_data, uncertainty, simulation)

In [6]:
smallness = TikhonovZero(n_params)

In [7]:
# Objective function
beta_0 = 1e4
regularization = beta_0 * smallness
phi = data_misfit + regularization

# Initial model
initial_model = np.zeros(n_params)

# Minimizer
minimizer = ConjugateGradient()

# Directives
directives = [
    MultiplierCooler(regularization, cooling_factor=2.0),
]

# Stopping criteria
chi_target = 0.1
stopping_criteria = [
    ChiTarget(data_misfit, chi_target=chi_target),
]

In [8]:
# Define inversion log
columns = {
    "iter": lambda iteration, _: iteration,
    "beta": lambda i, _: regularization.multiplier,
    "phi_d": lambda i, model: data_misfit(model),
    "phi_m": lambda i, model: regularization.function(model),
    "beta * phi_m": lambda i, model: regularization(model),
    "phi": lambda i, model: phi(model),
    "chi": lambda i, model: data_misfit(model) / data_misfit.n_data,
    "chi_target": lambda i, model: chi_target,
    "chi_target met?": lambda i, model: stopping_criteria[0](model),
}
inversion_log = InversionLog(columns)
inversion_log

In [9]:
inversion = Inversion(
    phi,
    initial_model,
    minimizer,
    directives=directives,
    stopping_criteria=stopping_criteria,
    cache_models=True,
    log=inversion_log,
)

In [10]:
inversion.run()

Output()

array([0.76518717, 0.6188566 , 0.24764406, 0.19526216, 0.35131399,
       0.19815277, 0.32923835, 0.27646319, 0.52978942, 0.86908809])

In [11]:
inversion.models

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0.07174391, 0.08166224, 0.07240393, 0.07703917, 0.07525462,
        0.08184415, 0.07872014, 0.07599201, 0.08152992, 0.09089636]),
 array([0.12272358, 0.13881626, 0.12221181, 0.12958872, 0.1276455 ,
        0.13812564, 0.13305127, 0.1283115 , 0.13878591, 0.15522723]),
 array([0.19138251, 0.21390537, 0.18574098, 0.19562915, 0.19585064,
        0.20986641, 0.20275434, 0.19514885, 0.21441523, 0.24130635]),
 array([0.26929379, 0.29449301, 0.24905953, 0.25897566, 0.26736706,
        0.28125271, 0.27332325, 0.26209185, 0.29646101, 0.33758253]),
 array([0.34751719, 0.36632983, 0.29516593, 0.2998452 , 0.3271845 ,
        0.33291987, 0.32733705, 0.31176376, 0.3708746 , 0.43137687]),
 array([0.4252272 , 0.42458387, 0.31496551, 0.30773535, 0.36782439,
        0.35423014, 0.35634832, 0.335552  , 0.43149642, 0.52004177]),
 array([0.50655047, 0.47343654, 0.31003452, 0.28628234, 0.38918978,
        0.34564177, 0.36309911, 0.33620328, 0.47822531

In [12]:
inversion_log.table

┏━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ iter ┃ beta     ┃ phi_d    ┃ phi_m    ┃ beta * phi_m ┃ phi      ┃ chi      ┃ chi_target ┃ chi_target met? ┃
┡━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0    │ 1.00e+04 │ 4.28e+03 │ 0.00e+00 │ 0.00e+00     │ 4.28e+03 │ 1.71e+02 │ 1.00e-01   │ False           │
│ 1    │ 1.00e+04 │ 2.90e+03 │ 6.22e-02 │ 6.22e+02     │ 3.52e+03 │ 1.16e+02 │ 1.00e-01   │ False           │
│ 2    │ 5.00e+03 │ 2.10e+03 │ 1.79e-01 │ 8.95e+02     │ 3.00e+03 │ 8.40e+01 │ 1.00e-01   │ False           │
│ 3    │ 2.50e+03 │ 1.26e+03 │ 4.21e-01 │ 1.05e+03     │ 2.31e+03 │ 5.03e+01 │ 1.00e-01   │ False           │
│ 4    │ 1.25e+03 │ 6.11e+02 │ 7.84e-01 │ 9.80e+02     │ 1.59e+03 │ 2.45e+01 │ 1.00e-01   │ False           │
│ 5    │ 6.25e+02 │ 2.55e+02 │ 1.18e+00 │ 7.36e+02     │ 9.91e+02 │ 1.02e+01 │ 1.00e-01   │ False           │
│ 6    │ 3.12e+02 │ 1.02e+02 │ 1.51e+00 │ 4.72e+02     │ 5.75e+02 │ 4.10e+00 │ 1.00e-01   │ False           │
│ 7    │ 1.56e+02 │ 4.29e+01 │ 1.77e+00 │ 2.77e+02     │ 3.20e+02 │ 1.71e+00 │ 1.00e-01   │ False           │
│ 8    │ 7.81e+01 │ 1.81e+01 │ 1.99e+00 │ 1.56e+02     │ 1.74e+02 │ 7.24e-01 │ 1.00e-01   │ False           │
│ 9    │ 3.91e+01 │ 7.19e+00 │ 2.18e+00 │ 8.53e+01     │ 9.24e+01 │ 2.88e-01 │ 1.00e-01   │ False           │
│ 10   │ 1.95e+01 │ 2.71e+00 │ 2.34e+00 │ 4.57e+01     │ 4.84e+01 │ 1.08e-01 │ 1.00e-01   │ False           │
│ 11   │ 9.77e+00 │ 1.09e+00 │ 2.45e+00 │ 2.39e+01     │ 2.50e+01 │ 4.37e-02 │ 1.00e-01   │ True            │
└──────┴──────────┴──────────┴──────────┴──────────────┴──────────┴──────────┴────────────┴─────────────────┘

In [13]:
import pandas as pd

df = pd.DataFrame(inversion_log.log).set_index("iter")
df

,beta,phi_d,phi_m,beta * phi_m,phi,chi,chi_target,chi_target met?
iter,,,,,,,,
0,10000.000000,4281.956360,0.000000,0.000000,4281.956360,171.278254,0.1,False
1,10000.000000,2901.066262,0.062236,622.359176,3523.425438,116.042650,0.1,False
2,5000.000000,2100.984190,0.178956,894.777818,2995.762008,84.039368,0.1,False
3,2500.000000,1257.260727,0.420950,1052.374179,2309.634906,50.290429,0.1,False
4,1250.000000,611.363211,0.784181,980.226728,1591.589939,24.454528,0.1,False
5,625.000000,255.114132,1.177861,736.163317,991.277448,10.204565,0.1,False
6,312.500000,102.392487,1.511980,472.493629,574.886116,4.095699,0.1,False
7,156.250000,42.856648,1.772643,276.975455,319.832103,1.714266,0.1,False
8,78.125000,18.088145,1.990755,155.527715,173.615859,0.723526,0.1,False


## Using constructor

In [14]:
uncertainty = 1e-2 * maxabs * np.ones_like(synthetic_data)
simulation = LinearRegressor(X)
data_misfit = DataMisfit(synthetic_data, uncertainty, simulation)

In [15]:
smallness = TikhonovZero(n_params)

In [16]:
starting_beta = 1e4
initial_model = np.zeros(n_params)
minimizer = ConjugateGradient()
chi_target = 0.1

inversion = create_inversion(
    data_misfit,
    smallness, 
    starting_beta=1e4,
    initial_model=initial_model,
    optimizer=minimizer,
    chi_target=chi_target,
)

We can inspect the inversion

In [17]:
inversion.directives

In [18]:
inversion.initial_model

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
inversion.objective_function

φ(m) + 1.00e+04 φ(m)

In [20]:
inversion.log.log

{'iter': [], 'beta': [], 'phi_d': [], 'phi_m': [], 'phi': [], 'chi': []}

In [21]:
inverted_model = inversion.run()

Output()

In [22]:
print("True model:")
print(true_model)
print("\nInverted model:")
print(inverted_model)

True model:
[0.78225148 0.67148671 0.2373809  0.17946133 0.34662367 0.15210999
 0.31142952 0.23900652 0.54355731 0.91770851]

Inverted model:
[0.76518717 0.6188566  0.24764406 0.19526216 0.35131399 0.19815277
 0.32923835 0.27646319 0.52978942 0.86908809]


In [23]:
inversion.model is inverted_model

True

In [24]:
inversion.objective_function

φ(m) + 9.77 φ(m)

In [25]:
len(inversion.models)

12

In [26]:
inversion.models[11]

array([0.76518717, 0.6188566 , 0.24764406, 0.19526216, 0.35131399,
       0.19815277, 0.32923835, 0.27646319, 0.52978942, 0.86908809])